# nnU-Net V2 Workshop 
The nnU-Net is a toolkit to segment imaging data and was specifically designed for biomedical data.

It was designed by **Fabian Isensee** while pursuing his PhD at the [Medical Image Computing Division of the German Cancer Research Institute](https://www.dkfz.de/en/mic/index.php).

More information about nnU-Net, as well as the installataion guide on your own workstation can be found on:
- Isensee, F., Jaeger, P. F., Kohl, S. A. A., Petersen, J. & Maier-Hein, K. H. [nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation](http://www.nature.com/articles/s41592-020-01008-z). Nat Methods 18, 203–211 (2021).
- [nnU-Net GitHub Page](https://github.com/MIC-DKFZ/nnUNet)


## What is this Workshop all about?

The goal of this workshop is to allow end-users to understand in how to use nnU-Net version 2 for their own applications.

It includes initial setup and how to use nnU-Net in the Google Colab environment. 
Some steps are only needed for GoogleColab and will be explained, while others are generally applicable. 

Further it is advised to visualize the data with segmentations using the [MITK-Workbench](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)), which is also developed in the Medical Image Computing Division of the German Cancer Research Institute - at least this is what we advocate.

- [MITK Workbench Download Link](https://www.mitk.org/wiki/Downloads)

### Requirements for Google Colab 
1. Google Account
   1. At least 4GB of free disk space
2. Visualization tool for volumetric data, we advise to use MITK.

### Some notes about the Workshop

This workshop was created by **Carsten Lüth**, a PhD student in the [Interactive Machine Learning Research Group](https://www.dkfz.de/en/interaktives-maschinelles-lernen/index.php) headed by Dr. Paul F. Jäger.

If you happen to find this workshop in any way useful, please share this version of it:
- [https://github.com/IML-DKFZ/nnunet-workshop](https://github.com/IML-DKFZ/nnunet-workshop)

## Further information

Further Information about nnU-Net is presented in the following lectures:
- [Lecture by Paul F. Jäger](https://www.youtube.com/watch?v=3po8qVzz5Tc&t=2196s)
- [Lecture by Fabian Isensee](https://www.youtube.com/watch?v=C6tpnJRpt90)

### MITK with nnU-Net
MITK has a new experimental feature which uses pretrained nnU-Net models as a segmentation tool.
This feature has been added since its release of version 2022.04.
For further information about this we refer to the [MITK Documentation](https://docs.mitk.org/2022.04/org_mitk_views_segmentation.html#org_mitk_views_segmentationnnUNetTool).


**Notes**:
- quite some time was spent to optimize the way of downloading and unpacking the data. I did not get to run it at all when automated... If you have any idea how to do this better (and show that it works I would greatly appreciate this!)

# 1. Import Packages for Google Colab

In [ ]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [ ]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

# 2. Installing nnU-Net

In [ ]:
# install nnunet - yes it is that easy
!pip install nnunetv2


     |████████████████████████████████| 251 kB 12.6 MB/s 
     |████████████████████████████████| 151 kB 45.1 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 2.0 MB 44.8 MB/s 
  Created wheel for nnunet: filename=nnunet-1.7.0-py3-none-any.whl size=487821 sha256=13eaebaba6a6193002385a2d21b302108b66247448de3802584ed33efd4089aa
  Stored in directory: /root/.cache/pip/wheels/66/f6/19/c51389976a9ae30212178d1c41497893504b900f043fe1c665
  Created wheel for batchgenerators: filename=batchgenerators-0.23-py3-none-any.whl size=84779 sha256=d5d6bfedfc9240e0afa65847249ada5cfd668f0317fc89fd220109696d6d25ea
  Stored in directory: /root/.cache/pip/wheels/81/3d/a2/9a602ce56458c684d2962ec89eddd6bc06b00687735e2dd11a
  Created wheel for dicom2nifti: filename=dicom2nifti-2.3.0-py3-none-any.whl size=42932 sha256=becb646cdb13e8c24abfc4f47d49effc82562dfe92922bb2e94c7a8a2d0d9364
  Stored in directory:

**Note**: you do not have to restart the Runtime even when the following error appears, this is simply due to a reinstallation of a package - so no worries. 


```
WARNING: The following packages were previously imported in this runtime:

[argparse]

You must restart the runtime in order to use newly installed versions.
```

# 3. Verifying installation of nn-Unet

In [ ]:
# check if nnunet can be imported
import nnunetv2




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



You should read the following if the installation was successful:


```
Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

```



### 3.1 Installation of GDown
GDown downloads files from GoogleDrive with Python.

In [ ]:
# install gdown to download files from GoogleDrive
!pip install gdown

# 4. Connect Google Colab with GoogleDrive
This is heavily encouraged when using Google Colab, otherwise loading, accessing and saving of data checkpoints etc. is just much harder (if not impossible).

**Note**:
When ```drive.mount()``` is executed, there will appear a popup window, which will ask you which Google Account should be connected and then for permission regarding Colab accessing your GoogleDrive.

There can be some problems at this step, for me the solution was to Download the GoogleDrive Application.
You can download it from: https://www.google.com/drive/download/

In [ ]:
# for colab users only - mounting the drive

from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

# 5. Setting up nnU-Nets folder structure and environment variables
nnUnet expects a certain folder structure and environment variables. 

Roughly they tell nnUnet:
1. Where to look for stuff
2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

## 5.1 Set environment Variables and creating folders

In [ ]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw" : os.path.join(mount_dir, "nnUNet_raw"), 
    "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_results" : os.path.join(mount_dir, "nnUNet_results"),
    "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), # This is used here only for convenience (not necessary for nnU-Net)!
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw_data_base exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder exists.
/content/drive/My Drive/Colab Notebooks/RawData exists.
If No Error Occured Continue Forward. =)


# 6. Using nnU-Net on Medical Decathlon datasets
To get us started and also verify that nnU-Net works properly on our system, we will now run a training with nnU-Net and then make some predictions.

This will be done on parts of the Medical Decathlon Dataset, because everything is already nicely set up for these datasets.

Also for these two datasets there exist pre-written scripts, automatically unpacking and preparing the data for nnU-Net to use.

For this we will:
- Train on the hippocampus dataset
- Run Inference on the prostate dataset

Bottom Line:
If something goes wrong here, it is worth to check whether anything went wrong before these steps! 

## 6.1 Downloading the Data
Download the data, unpack it and put it in the proper folder.
Generally, you will then have to download and move the files to the correct folder yourself.

Link to official [GoogleDrive](https://drive.google.com/drive/folders/1HqEgzS8BV2c7xYNrZdEAnrHk7osJJ--2) for the download.

### 6.1.1 Downloading the data

In [ ]:
os.chdir(path_dict["RAW_DATA_PATH"])
# Download the Hippocampus Dataset
!gdown 'https://drive.google.com/file/d/1k4mgt1L-RznfAErdaHjONXixnpHe85EP'

# Download the Prostate Dataset
!gdown  'https://drive.google.com/file/d/1IRBf1Q7zJWwlS7TNyxoH1zp3HzAuIBRH'
os.chdir(base_dir)

In [ ]:
print("Data should be located in folder: {}".format(path_dict["RAW_DATA_PATH"]))
assert os.path.isfile(os.path.join(path_dict["RAW_DATA_PATH"], "Task04_Hippocampus.zip")) # check whether the file is correctly downloaded
assert os.path.isfile(os.path.join(path_dict["RAW_DATA_PATH"], "Task05_Prostate.zip")) # check whether the file is correctly downloaded


In [ ]:
# unzip the files in the folder
os.chdir(path_dict["RAW_DATA_PATH"])
!unzip -o Task04_Hippocampus.zip
!unzip -o Task05_Prostate.zip
!rm Task04_Hippocampus.zip
!rm Task05_Prostate.zip
os.chdir(base_dir)

### 6.1.2 Verification of data
Here we test, whether the data is saved in the correct folder structure.

In [ ]:
print("Data should be located in folder: {}".format(path_dict["RAW_DATA_PATH"]))
assert os.path.exists(os.path.join(path_dict["RAW_DATA_PATH"], "Task04_Hippocampus")) # check whether the file is correctly downloaded
assert os.path.exists(os.path.join(path_dict["RAW_DATA_PATH"], "Task05_Prostate")) # check whether the file is correctly downloaded

## 6.2 Training nnU-Net on the Decathlon Hippocampus Dataset
For this we will use already pre-existing scripts for handling the data.

### 6.2.1 Dataset Conversion
The Decathlon datasets are 4D nifti files, for nnU-Net they have to be converted to 3D nifti files.

For more information about dataset conversion see: [nnU-Net Dataset Formatting Instructions](https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/dataset_format.md)

In [ ]:
!nnUNetv2_convert_MSD_dataset -i "${RAW_DATA_PATH}/Task04_Hippocampus"



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



## 6.2.2 Extracting Rule Based Parameters
This will preprocess the dataset to allow fast training and saves it into the "nnUNet_preprocessed" folder.
Further rule based parameters will be extracted in the planning step.

In [ ]:
# Prepare the Execution of nnU-Net for Dataset 4 - this is the Hippocampus Dataset here (taking 1-2 minutes)
!nnUNetv2_plan_and_preprocess -d 4



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

hippocampus_367
hippocampus_189
hippocampus_185
hippocampus_294
hippocampus_064
hippocampus_150
hippocampus_314
hippocampus_107
hippocampus_243
hippocampus_304
hippocampus_099
hippocampus_007
hippocampus_343
hippocampus_050
hippocampus_023
hippocampus_332
hippocampus_204
hippocampus_269
hippocampus_353
hippocampus_296
hippocampus_220
hippocampus_105
hippocampus_394
hippocampus_232
hippocampus_279
hippocampus_277
hippocampus_253
hippocampus_288
hippocampus_178
hippocampus_097
hippocampus_123
hippocampus_298
hippocampus_318
hippocampus_308
hippocampus_222
hippocampus_042
hippocampus_166
hippocampus_320
hippocampus_176
hippocampus_15

### 6.2.3 Training nnU-Net
here we will train a 3D nnU-Net on Full Resolution for 2 epochs.

To run a normal training use: ```nnUNetTrainer``` instead of ```nnUNetTrainer_1epoch```

In [ ]:
# train the 3d nnUnet on the Full Resolution with Dataset 4 and Cross Validation Split 0
!nnUNetv2_train 4 3d_fullres 0 -tr nnUNetTrainer_1epoch 

# Training for 2 iterations will take 



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  2
modalities:  {0: 'MRI'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT')])
stages...

stage:  0
{'batch_size': 9, 'num_pool_per_axis': [3, 3, 3], 'patch_size': array([40, 56, 40]), 'median_patient_size_in_voxels': array([36, 50, 35]), 'current_spaci

## 6.3 Inference with Pre-Trained Models on Decathlon Prostate Dataset
Here we will use a pre-trained model on the Prostate Dataset and visualize the results

### 6.3.1 Downloading Pre-trained Model
First download the pre-trained models and validate that everything works correctly 

In [ ]:
# Download the Pretrained Model for the Prostate Dataset (Decathlon Dataset)
os.chdir(path_dict["nnUNet_results"])
!gdown 'https://drive.google.com/uc?export=download&id=1gCdlj-O3hgoMDDw2NM8QxvFIGGVbT9Qy'
!nnUNetv2_install_pretrained_model_from_zip nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip
!rm 'nnUNetv2_3dfullres_Model_Dataset005_Prostate.zip'
os.chdir(base_dir)

# takes roughly 9 minutes



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet


######################################################
!!!!!!!!!!!!!!!!!!!!!!!!WARNING!!!!!!!!!!!!!!!!!!!!!!!
######################################################
Using the pretrained model weights is subject to the license of the dataset they were trained on. Some allow commercial use, others don't. It is your responsibility to make sure you use them appropriately! Use nnUNet_print_pretrained_model_info(task_name) to see a summary of the dataset and where to find its license!
######################################################

Download finished. Extracting...
Done


### 6.3.2 Convert Prostate Dataset
For inference the data has to be saved into the format nnU-Net expects as input (as detailed in 6.2.1).

In [ ]:
# Preprocess Prostate Dataset
!nnUNetv2_convert_MSD_dataset -i "${RAW_DATA_PATH}/Task05_Prostate"



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



### 6.3.3 Inference on the Prostate Dataset
Use 3d_fullres nnUnet (-c) trained on the test set of dataset 5 (-d 5) and save the outputs

In [ ]:
# use fully trained nnU-Net to make predictions on data
!nnUNetv2_predict -i "${nnUNet_raw}/Dataset005_Prostate/imagesTs/" -o "${nnUNet_results}/Dataset005_Prostate/predTs/" -d 5 -c 3d_fullres



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder/nnUNet/3d_fullres/Task005_Prostate/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 16 unique case ids, here are some examples: ['prostate_36' 'prostate_09' 'prostate_12' 'prostate_27' 'prostate_30'
 'prostate_26' 'prostate_08' 'prostate_27' 'prostate_12' 'prostate_36']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 16
number of cases that still need to be predicted: 0
emptying cuda cache
loading parameters for folds, None
folds is 

## 6.3.4 Visualization of Data and Segmentations
Download files from Google Drive:

Images from: ```${nnUNet_raw}/Dataset005_Prostate/imagesTs/```

Segmentations from: ```${RESULTS_FOLDER}/Dataset005_Prostate/predTs/"```


After downloading these files you can visualize them with any volumetric visualization program.
For this we would advise to use [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK)) which already has some great [tutorials](https://www.mitk.org/wiki/Tutorials).


Note:
- If you have not already downloaded it, here is the [MITK Download Link](https://www.mitk.org/wiki/Downloads)
